In [3]:
import gym
from gym import envs
from gym import wrappers

import numpy as np
import sys
from collections import defaultdict

if "../" not in sys.path:
  sys.path.append("../")

import plotting

import matplotlib
matplotlib.style.use('ggplot')

In [4]:
def epsilon_greedy_policy(Q, state, num_actions, epsilon):
    policy = np.ones(num_actions, dtype=float) * epsilon / num_actions
    best_action = np.argmax(Q[state])
    policy[best_action] += 1-epsilon
    return policy

In [116]:
def learn(env_n, num_episodes,\
        outdir,\
        Q= defaultdict(lambda: np.zeros(env.action_space.n)),\
        discount_factor = 0.8,\
        lambda_factor = 0.9,\
        replacing_traces=True,\
        folder_options = '',\
        delete_content=False,\
        monitor=False):
    envi = gym.make(env_n)
    outdir = outdir + env_n + folder_options + "/"
    global env
    env = (wrappers.Monitor(envi, outdir, video_callable=False, force=delete_content) if monitor else envi)
    
    er = np.zeros(num_episodes)
    el = np.zeros(num_episodes)
    
    for i_episode in range(1,num_episodes+1):
        if (i_episode%1000)==0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        #initialization: use a list of (state, action) pair to keep
        #track of the episode "history"
        episode = []
        E = defaultdict(lambda: np.zeros(env.action_space.n))
        alpha = 1 - i_episode/num_episodes #0.04
        epsilon = (1 - i_episode/num_episodes) ** 2
        #initial state
        state = env.reset()
        #calculate the e-greedy policy for the init state given the Q function
        #and choose the action accordingly
        policy = epsilon_greedy_policy(Q, state, env.action_space.n, epsilon)
        action = np.random.choice(np.array(len(policy)), p=policy)
        while True:
            if ((state, action) not in episode):
                episode.append((state, action))
            next_state, reward, done, _= env.step(action)
            r = -1 if (done and reward==0) else reward
            policy = epsilon_greedy_policy(Q, next_state, env.action_space.n, epsilon)
            next_action = np.random.choice(np.array(len(policy)), p=policy)
            delta = r + discount_factor * Q[next_state][next_action] - Q[state][action]
            
            # Update statistics
            # Update statistics
            er[i_episode-1] += reward
            el[i_episode-1] += 1

            #updating the trace using the replacing or the eligibility
            if replacing_traces:
                E[state][action]  = 1
            else:
                E[state][action]  += 1

            #improvement of the Q value function for each state visited in the current episode
            for (states, actions) in episode:
                Q[states][actions] = Q[states][actions] + alpha * delta * E[states][actions]
                E[states][actions] = discount_factor * lambda_factor * E[states][actions]
                
            
            state = next_state; action = next_action;
            if done:
                break
    env.close()
    return Q, er, el

In [120]:
Q, er, el = learn(
    env_n='FrozenLake-v0',\
    num_episodes=1000,\
    outdir= "/Users/jacopo/openaigym/project/TD/results/SARSA_Lambda/",\
    replacing_traces=True,\
    folder_options = '1000',\
    delete_content=True,\
    monitor=True)

[2017-01-28 17:35:00,798] Making new env: FrozenLake-v0
[2017-01-28 17:35:00,809] DEPRECATION WARNING: env.spec.timestep_limit has been deprecated. Replace your call to `env.spec.timestep_limit` with `env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')`. This change was made 12/28/2016 and is included in version 0.7.0
[2017-01-28 17:35:00,811] Creating monitor directory /Users/jacopo/openaigym/project/TD/results/SARSA_Lambda/FrozenLake-v01000/


Episode 1000/1000.

[2017-01-28 17:35:01,766] Finished writing results. You can upload them to the scoreboard via gym.upload('/Users/jacopo/openaigym/project/TD/results/SARSA_Lambda/FrozenLake-v01000')


In [121]:
stats = plotting.calc_stats(er, el)
plotting.plot_episode_stats(stats)